In [ ]:
import face_recognition
import cv2
import numpy as np
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
dt.datetime.now().strftime("%x")

'01/02/23'

In [ ]:
dt.datetime.now().strftime("%X")

'23:31:51'

In [ ]:
# faceEncodings= pd.read_csv("known_faces_encoding.csv")
# faceNames = pd.read_csv("Known_names.csv")
# known_face_encodings = faceEncodings.values
# known_face_names = list(faceNames["name"])

In [ ]:
def register():
    new_name = input("Enter New Candidate : ")

    video_capture = cv2.VideoCapture(0)
    while True:
        _,frame = video_capture.read()


        z = face_recognition.face_locations(frame)
        for top, right, bottom, left in z:
            cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (255, 0, 0), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, new_name+" ?", (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        cv2.imshow("video",frame)


        if cv2.waitKey(1) & 0xFF == ord('s'):
            
            cv2.imwrite("new_pic.jpg",frame)
            new_image = face_recognition.load_image_file("new_pic.jpg")
            new_face_encoding = face_recognition.face_encodings(new_image)[0]
            temp = pd.DataFrame(new_face_encoding).T
            faceEncodings= pd.read_csv("known_faces_encoding.csv")
            faceEncodings.columns = pd.to_numeric(faceEncodings.columns)
            faceEncodings = faceEncodings.append(temp,ignore_index=True)

            faceEncodings.to_csv("known_faces_encoding.csv",index=False)
            faceNames = pd.read_csv("Known_names.csv")
            known_face_names = list(faceNames["name"])
            known_face_names.append(new_name)
            known_names = pd.DataFrame(data={"name":known_face_names})
            known_names.to_csv("Known_names.csv",index=False)
            print(f"{new_name} suscefully registered")

            break




        if cv2.waitKey(1) & 0xFF == ord('a'):
            break
    video_capture.release()
    cv2.destroyAllWindows()

In [ ]:
def delete(name):
    try:
        faceEncodings= pd.read_csv("known_faces_encoding.csv")
        faceNames = pd.read_csv("Known_names.csv")
        num=faceNames[faceNames["name"]==name].index[0]
        faceNames.drop(labels=num,inplace=True)
        faceNames.to_csv("Known_names.csv",index=False)
        faceEncodings.drop(labels=num,inplace=True)
        faceEncodings.to_csv("known_faces_encoding.csv",index=False)
        print(f"{name} deleted sucussfully")
    except:
        print("No Such Name")

In [ ]:
def attendence():
    faceEncodings= pd.read_csv("known_faces_encoding.csv")
    faceNames = pd.read_csv("Known_names.csv")
    known_face_encodings = faceEncodings.values
    known_face_names = list(faceNames["name"])
    video_capture = cv2.VideoCapture(0)
    face_locations = []
    face_encodings = []
    face_names = []                    
    process_this_frame = True

    while True:
        ret, frame = video_capture.read()
        if process_this_frame:
            small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
            rgb_small_frame = small_frame[:, :, ::-1]
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
            face_names = []
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]
                face_names.append(name)
        process_this_frame = not process_this_frame
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4
            if name != "Unknown":
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                db = pd.read_csv("attendence.csv")
                if name not in list(db[(db["date"]==dt.datetime.now().strftime("%x"))]["Person"]): 
                    temp = pd.DataFrame(data={"date":[dt.datetime.now().strftime("%x")],"time":[dt.datetime.now().strftime("%X")],"Person":[name]})
                    db = db.append(temp)
                    db.to_csv("attendence.csv",index=False)
                else:
                    font = cv2.FONT_HERSHEY_DUPLEX
                    cv2.putText(frame, name+" OK", (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
                    
                    
                
            else:
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        cv2.imshow('Video', frame)
        if cv2.waitKey(1) & 0xFF == ord('a'):
            break
    video_capture.release()
    cv2.destroyAllWindows()

In [ ]:
def get_details():
    faceEncodings= pd.read_csv("known_faces_encoding.csv")
    faceNames = pd.read_csv("Known_names.csv")
    known_face_encodings = faceEncodings.values
    known_face_names = list(faceNames["name"])
    print(len(known_face_encodings),(known_face_names))
    
def get_attendence():
    db = pd.read_csv("attendence.csv")
    return db


In [ ]:
attendence()

In [ ]:
register()

Enter New Candidate : king
king suscefully registered


In [ ]:
delete("Sharuk")

Sharuk deleted sucussfully


In [ ]:
get_details()

4 ['DEMO', 'DO NOT DELETE', 'Chetan', 'king']


In [ ]:
get_attendence()

,date,time,Person
0,01/02/23,23:36:03,Demo


In [ ]:
db = get_attendence()

In [ ]:
list(db[(db["date"]==dt.datetime.now().strftime("%x"))]["Person"])

['Chetan',
 'Chetan',
 'king',
 'king',
 'Chetan',
 'Chetan',
 'Chetan',
 'Chetan',
 'Chetan',
 'Chetan',
 'Chetan',
 'Chetan',
 'Chetan',
 'Chetan',
 'Chetan',
 'Chetan',
 'Chetan',
 'Chetan',
 'Chetan',
 'Chetan',
 'Chetan',
 'Chetan',
 'Chetan']

In [ ]:
dt.datetime.now().strftime("%x")

'01/04/23'